---------- Start of generic setup code to get kaggle credentials login and data to work with. Copied much from Workshop 2 ----------

In [2]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
!pip install -U -q scikeras
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

https://drive.google.com/file/d/1DeYgEzZ8Bs8Y3QsHd5YsV-3D-SzVAm4x

In [16]:
#GET data from file I've uploaded to my drive. Use your own kaggle.json and google auth logins etc
downloaded = drive.CreateFile({'id':'1DeYgEzZ8Bs8Y3QsHd5YsV-3D-SzVAm4x'}) 
#This file was created after much trial and error
#results = drive.CreateFile({'id':'1tWi1eK4K-kESDUD3VpGKe3UX83sj2XnJ'})
#downloaded.GetContentFile('XGB_Results.csv')
downloaded.GetContentFile('predict-energy-consumption.zip')

In [5]:
!unzip predict-energy-consumption.zip

!ls

Archive:  predict-energy-consumption.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
predict-energy-consumption.zip	sample_submission.csv  train.csv
sample_data			test.csv


In [6]:
from google.colab import files

files.upload(); # upload your kaggle.json file

Saving kaggle.json to kaggle.json


Processing data from the source.

In [7]:
import json

!mkdir /root/.kaggle/
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [8]:
#GENERIC SUBMIT FUNCTION (uncomment when you want to submit)

#submission = pd.DataFrame(
#    {'key': test.key, 'fare_amount': val_predictions},
#    columns = ['key', 'fare_amount'])
#submission.to_csv('submission.csv', index = False)


#!kaggle competitions submit -c predict-electricity-consumption -f submission.csv -m "[TEAM NAME] Submission"

In [9]:
#------------------ ACTUAL PROBLEM SOLVING BELOW ---------------------------

import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
 
#Here we can split into train/test for verification while we train whatever works)
train =  pd.read_csv('train.csv')
train_labels = train['consumption'] #y vals in this case are concumption

#GIVEN time is a string we just gonna drop that mfer, cbf processing it etc
train = train.drop(['time'], 1)
train = train.drop(['consumption'], 1)
#print(train)

test = pd.read_csv('test.csv')
test = test.drop(['time'],1)
#Have to predict consumption for these values. Consumption unkown.

#print(train)
#print(test)

#At this point should have a train and test without Nan values:)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


---------- End of generic setup code. Actual programming can begin after here ----------

In [10]:
def replaceNanWithZero(test, train, labels):
  train_labels = labels.replace(np.nan, 0.0)

  for col in train.columns:
    train[col] = train[col].replace(np.nan, 0.0)

  for col in test.columns:
    test[col] = test[col].replace(np.nan, 0.0)

  return test, train, train_labels

In [11]:
#PCA to get a lesser number of features, then going to feed into basic nn see what we can do
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def PCA_Performance(train, test):
  variance = [0.5,0.6,0.75,0.8,0.85,0.9,0.95,0.96,0.97,0.98,0.99]

  for var in variance:
    pca = PCA(n_components = var) 
    pca.fit(train)
    transformed_Train = pca.transform(train)
    #transformed_Test = pca.transform(test)
    print(f'Retained {var} of variance with {len(transformed_Train.T)} number of features')

In [12]:
#USE the code we wrote above
test_, train, labels = replaceNanWithZero(test, train, train_labels)

PCA_Performance(train, test)

Retained 0.5 of variance with 1 number of features
Retained 0.6 of variance with 1 number of features
Retained 0.75 of variance with 1 number of features
Retained 0.8 of variance with 1 number of features
Retained 0.85 of variance with 1 number of features
Retained 0.9 of variance with 2 number of features
Retained 0.95 of variance with 3 number of features
Retained 0.96 of variance with 4 number of features
Retained 0.97 of variance with 4 number of features
Retained 0.98 of variance with 5 number of features
Retained 0.99 of variance with 5 number of features


XGBoost Model: Train a default version, then use hyperopt to tweak it to find best hyperparam values.

In [13]:
!pip install hyperopt

#setup etc
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
import random

results = {}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
#NOTE: http://learningsys.org/papers/LearningSys_2015_paper_32.pdf
#sklearn GradientBoostRegressor fundamentally the same as XGBoost, just much slower
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.25, random_state=8080)

xgb = GradientBoostingRegressor()

xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)

score = mean_squared_error(pred,y_test,squared = True)

score
#TODO: okay i mean default xgb pretty epic

1.0030915328208518

In [15]:
def objective(search_space):
    model = GradientBoostingRegressor(**search_space)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = mean_squared_error(y_test, y_pred, squared=True)
    return {'loss': accuracy, 'status': STATUS_OK}

search_space={'loss':hp.choice('loss',['squared_error','absolute_error','huber','quantile']),
              'learning_rate':hp.uniform('learning_rate',0.0,1.0),
              'n_estimators':hp.choice('n_estimators',list(range(1,500))),
              'subsample':hp.uniform('subsample',0.0,0.99),
              'criterion':hp.choice('criterion',['squared_error','friedman_mse']),
              'min_samples_split':hp.choice('min_samples_split', list(range(2,500))),
              'min_samples_leaf':hp.choice('min_samples_leaf', list(range(1,500))), 
              'min_weight_fraction_leaf':hp.uniform('min_weight_fraction_leaf', 0.0,0.5),
              'max_depth':hp.choice('max_depth', list(range(1,100))),
              'min_impurity_decrease':hp.uniform('min_impurity_decrease', 0.0,10.0)}
              #there are some other params but they seem largely irrelevant}           
              #'shrinking':hp.choice('shrinking',[True,False])}
algorithm=tpe.suggest

results = []

#gimme gimme 
for i in range(0,5):
  trials =Trials()
  best_params = fmin(fn=objective,space=search_space,algo=algorithm,max_evals=2000,trials=trials)
  best_params['best_loss'] = min(trials.losses())
  results.append(best_params)

  0%|          | 4/2000 [00:04<34:47,  1.05s/it, best loss: 1.2278513638515596]


KeyboardInterrupt: ignored

In [ ]:
#so I don't have to rerun the above code every time ^
from google.colab import files

df = pd.DataFrame(results)
df.to_csv('results.csv') 
files.download('results.csv')

In [33]:
def convert_to_params(instance):
  #item['criterion'] = int(item['criterion'])
  #item['loss'] = int(item['loss'])
  crit = int(instance['criterion'])
  if crit == 0:
    instance.loc['criterion'] = 'squared_error'
  elif crit == 1:
    instance.loc['criterion'] = 'friedman_mse'
  
  los = int(instance['loss'])
  if los == 0:
    instance.loc['loss'] = 'squared_error'
  elif los == 1:
    instance.loc['loss'] = 'absolute_error'
  elif los == 2:
    instance.loc['loss'] = 'huber'
  elif los == 3:
    instance.loc['loss'] = 'quantile'
  
  instance['max_depth'] = int(instance['max_depth'])
  instance['min_samples_leaf'] = int(instance['min_samples_leaf'])
  instance['min_samples_split'] = int(instance['min_samples_split'])
  instance['n_estimators'] = int(instance['n_estimators'])
  return instance

In [36]:
res = pd.read_csv('XBG_Results.csv')

res2 = res.drop('Unnamed: 0',axis=1)
res2.sort_values('best_loss', inplace=True)
res3 = res2.drop('best_loss',axis=1)
best = res3[:5]

#print(best)

    criterion  learning_rate  loss  max_depth  min_impurity_decrease  \
26          1       0.040444     0         51               0.302976   
28          1       0.018668     0         22               0.175736   
29          1       0.069602     0         68               0.006988   
27          1       0.064183     0         70               1.053356   
25          1       0.112837     0         58               0.729574   

    min_samples_leaf  min_samples_split  min_weight_fraction_leaf  \
26                 4                  4                  0.000577   
28                11                 35                  0.000139   
29                12                 87                  0.000269   
27                14                 55                  0.000420   
25                21                 69                  0.000651   

    n_estimators  subsample  
26            94   0.844265  
28           289   0.972727  
29           304   0.979774  
27           175   0.848388  
25

In [39]:
from numpy.lib.npyio import savez_compressed
models = []
scores = {}
for index, row in best.iterrows():
  params = convert_to_params(row)
  model = GradientBoostingRegressor(**params)
  models.append(model)
  scores[model] = []

#we have the five best models. Train each over five random train/test splits and 
#pick one with lowest general sum we do this because num_iterations for HyperOpt 
#was very high, so want to avoid any potential overiftting
for i in range(0,5):
    print(i)
    X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.2)

    for model in models:
      model.fit(X_train,y_train)
      pred = model.predict(X_test)
      score = mean_squared_error(pred,y_test,squared = True)
      scores[model].append(score)

In [43]:
bscore = 100000
best = None
for key in scores.keys():
  thisScores = scores[key]
  score = np.sum(thisScores)
  if score < bscore:
    bscore = score
    best = key

print(best)

GradientBoostingRegressor(learning_rate=0.0404442995899507, max_depth=51,
                          min_impurity_decrease=0.3029757517818807,
                          min_samples_leaf=4, min_samples_split=4,
                          min_weight_fraction_leaf=0.0005774056088614,
                          n_estimators=94, subsample=0.8442646226998577)
